# Exploring neural network modeling strategies

## Preprocessing the data

In [212]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [213]:
import os
import polars as pl
import numpy as np
from pathlib import Path
import kagglehub

from utils import (
    download_data,
)

from constants import (
    DATASET_PATH,
)

In [214]:
data_path = download_data()

data_path

Kaggle credentials set.


In [215]:
data_path = Path(DATASET_PATH)

data_path

PosixPath('data/problematic-internet-use')

In [216]:
train = pl.read_csv(data_path / 'train.csv')

In [217]:
features = train.columns[:-1]
label = train.columns[-1:]

X = train[features]
y = train[label]

In [218]:
features_categorical = [
    'Basic_Demos-Enroll_Season',
    'CGAS-Season',
    'Physical-Season',
    'Fitness_Endurance-Season',
    'FGC-Season',
    'FGC-FGC_CU_Zone',
    'FGC-FGC_GSND_Zone',
    'FGC-FGC_GSD_Zone',
    'FGC-FGC_PU_Zone',
    'FGC-FGC_SRL_Zone',
    'FGC-FGC_SRR_Zone',
    'FGC-FGC_TL_Zone',
    'BIA-Season',
    'BIA-BIA_Activity_Level_num',
    'BIA-BIA_Frame_num',
    'PAQ_A-Season',
    'PAQ_C-Season',
    'PCIAT-Season',
    'PCIAT-PCIAT_01',
    'PCIAT-PCIAT_02',
    'PCIAT-PCIAT_03',
    'PCIAT-PCIAT_04',
    'PCIAT-PCIAT_05',
    'PCIAT-PCIAT_06',
    'PCIAT-PCIAT_07',
    'PCIAT-PCIAT_08',
    'PCIAT-PCIAT_09',
    'PCIAT-PCIAT_10',
    'PCIAT-PCIAT_11',
    'PCIAT-PCIAT_12',
    'PCIAT-PCIAT_13',
    'PCIAT-PCIAT_14',
    'PCIAT-PCIAT_15',
    'PCIAT-PCIAT_16',
    'PCIAT-PCIAT_17',
    'PCIAT-PCIAT_18',
    'PCIAT-PCIAT_19',
    'PCIAT-PCIAT_20',
    'SDS-Season',
    'PreInt_EduHx-Season',
    'PreInt_EduHx-computerinternet_hoursday'
]

### Droping `id` column

In [219]:
X = X.drop(['id'])

features = X.columns

### Droping unlabeled data:

In [220]:
y.to_series().value_counts()

sii,count
i64,u32
null,1224
1,730
0,1594
2,378
3,34


In [221]:
data = pl.concat([X, y], how='horizontal')
data = data.lazy().filter(pl.col(label).is_not_null()).collect()

X = data[features]
y = data[label]

In [222]:
y.to_series().value_counts()

sii,count
i64,u32
2,378
3,34
1,730
0,1594


### Add _one-hot encodings_ for the categorical columns and drop them:

In [224]:
one_hot = X.select(pl.col(features_categorical)).to_dummies()
features_one_hot = one_hot.columns

X = pl.concat([X, one_hot], how='horizontal')
X = X.drop(features_categorical)

features = X.columns

ColumnNotFoundError: Basic_Demos-Enroll_Season

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'select' <---
DF ["Basic_Demos-Age", "Basic_Demos-Sex", "CGAS-CGAS_Score", "Physical-BMI"]; PROJECT */270 COLUMNS; SELECTION: None

There is a lot of missing data, let's use the [`KNNImputer` class](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html) to fill them (see [this guide](https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/)):

In [203]:
size = X.height * X.width
missing = sum(X.null_count().row(0))

print(f'We have {size} total cells and {missing} missing cells ({100 * missing / size:.2f}% of data)')

We have 703858 total cells and 34862 missing cells (4.95% of data)


In [204]:
from sklearn.impute import KNNImputer

imputer = KNNImputer()
imputer.fit(X.to_numpy())
X_trans = imputer.transform(X.to_numpy())

Re-creating the `X` dataframes:

In [205]:
X = pl.DataFrame(schema=X.schema, data=X_trans)

In [206]:
size = X
missing = sum(X.null_count().row(0))

print(f'We have {size} total cells and {missing} missing cells ({100 * missing / size:.2f}% of data)')

We have 738720 total cells and 0 missing cells (0.00% of data)
